In [148]:
import numpy as np
import pandas as pd
import re
import os 
import random 
import spacy
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding
from keras.utils import to_categorical
from pickle import dump, load


df = pd.read_csv("./tweets_cunha.csv",encoding='utf-8')
df.head(5)
# Not gonna bother cleaning up the data because i'm only interested in the text colummn

,Unnamed: 0,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL
0,0,DeputadoEduardoCunha,@DepEduardoCunha,2014-01-06T17:47:42.000Z,"Eduardo Cunha | Blog | Blatter, a razão e a 'm...",NaN,NaN,NaN,NaN,NaN,[],https://twitter.com/DepEduardoCunha/status/420...
1,1,Lucio Vieira Lima,@luciovl_,2014-01-10T12:46:55.000Z,Replying to \r\n@DepEduardoCunha,@DepEduardoCunha\r\n Bom dia!,NaN,1.0,NaN,NaN,[],https://twitter.com/luciovl_/status/4216241920...
2,2,DeputadoEduardoCunha,@DepEduardoCunha,2014-01-10T13:12:32.000Z,@luciovl_\r\n @DepEduardoCunha\r\n bom dia,NaN,NaN,3.0,NaN,NaN,[],https://twitter.com/DepEduardoCunha/status/421...
3,3,DeputadoEduardoCunha,@DepEduardoCunha,2014-01-10T12:44:07.000Z,"Eduardo Cunha | Blog | Presí­dios brasileiros,...",NaN,NaN,3.0,3,1,[],https://twitter.com/DepEduardoCunha/status/421...
4,4,DeputadoEduardoCunha,@DepEduardoCunha,2014-01-05T17:42:14.000Z,"Boa tarde a todos,ano mal começou e já está ag...",NaN,NaN,5.0,196,66,[],https://twitter.com/DepEduardoCunha/status/419...


# Tokenizing the data and Training the model

In [76]:
# Downloading trained pipelines for Portuguese if its not installed already
# !python -m spacy download pt_core_news_lg

nlp = spacy.load('pt_core_news_lg',disable=['parser', 'tagger','ner'])

In [78]:
doc_text = ''.join(str(df["Text"].tolist())

trash = '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n\r\n~" ' # Stripping tweets
tokens = [re.sub(r'^https?:\/\/.*[\r\n]*', '', token.text.lower(), flags=re.MULTILINE) for token in nlp(doc_text) if token.text not in trash]

tokens = separate_punc(txt)
l_ats = ['\\r\\n@depeduardocunha','@depeduardocunha\\r\\n','\\r\\n@camaradeputados'] # Removing common @s that cause a lot of bias in the final result
tokens = list(filter(lambda a: a not in l_ats, tokens))
len(tokens)
tokens

['eduardo',
 'cunha',
 'blog',
 'blatter',
 'a',
 'razão',
 'e',
 'a',
 "'",
 'malandragem',
 "'",
 '',
 '…',
 'via',
 "'",
 'replying',
 'to',
 "'",
 "'",
 '@luciovl_\\r\\n',
 'bom',
 'dia',
 "'",
 "'",
 'eduardo',
 'cunha',
 'blog',
 'presí\\xaddios',
 'brasileiros',
 'verdadeiras',
 'bombas-relógio',
 '',
 '…',
 'via',
 "'",
 "'",
 'boa',
 'tarde',
 'a',
 'todos',
 'ano',
 'mal',
 'começou',
 'e',
 'já',
 'está',
 'agitado',
 "'",
 "'",
 'não',
 'podemos',
 'ficar',
 'de',
 'cócoras',
 'para',
 'o',
 'judiciário',
 "'",
 'diz',
 'o',
 'líder',
 'do',
 'pmdb',
 'eduardo',
 'cunha',
 '',
 '…',
 'via',
 "'",
 'terça',
 'irei',
 'a',
 'brasília',
 'para',
 'cuidar',
 'dessas',
 'confusões',
 'de',
 'emendas',
 'dos',
 'deps',
 'da',
 'bancada',
 "'",
 "'",
 'ontem',
 'estive',
 'em',
 'palmas',
 'prestigiando',
 'o',
 'dep',
 'junior',
 'coimbra',
 'junto',
 'de',
 'vários',
 'deps',
 "'",
 "'",
 'o',
 'ano',
 'será',
 'de',
 'trabalho',
 'intenso',
 "'",
 "'",
 'amanha',
 'estarei',
 '

In [129]:
# organize into sequences of tokens
train_len = 25 

text_sequences = [tokens[i-train_len+1:i] for i in range(train_len+1, len(tokens))]

In [130]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

tokenizer.index_word

sequences = np.array(tokenizer.texts_to_sequences(text_sequences))
sequences

array([[  21,   14,  231, ...,    1,   21,   14],
       [  14,  231, 7029, ...,   21,   14,  231],
       [ 231, 7029,    2, ...,   14,  231, 2027],
       ...,
       [   4, 7024,    5, ...,    2,  174,  100],
       [7024,    5,    2, ...,  174,  100, 7030],
       [   5,    2,  192, ...,  100, 7030, 7031]])

In [131]:
vocab_sz = len(tokenizer.word_counts)

X = sequences[:,:-1]
y = to_categorical(sequences[:,-1], num_classes=vocab_sz+1)

In [132]:
# Implementing a LSTM model
model = Sequential()

model.add(Embedding(vocab_sz+1, train_len, input_length=train_len))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(150, activation='relu'))
model.add(Dense(vocab_sz+1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 25, 25)            175800    
_________________________________________________________________
lstm_8 (LSTM)                (None, 25, 150)           105600    
_________________________________________________________________
lstm_9 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_8 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_9 (Dense)              (None, 7032)              1061832   
Total params: 1,546,482
Trainable params: 1,546,482
Non-trainable params: 0
_________________________________________________________________


In [134]:
model.fit(X, y, batch_size=128, epochs=290, verbose=1)

Epoch 1/292
378/378 [==============================] - 36s 95ms/step - loss: 4.3997 - accuracy: 0.2087
Epoch 2/292
378/378 [==============================] - 38s 101ms/step - loss: 4.3031 - accuracy: 0.2117
Epoch 3/292
378/378 [==============================] - 38s 101ms/step - loss: 4.2094 - accuracy: 0.2176
Epoch 4/292
378/378 [==============================] - 38s 100ms/step - loss: 4.1154 - accuracy: 0.2214
Epoch 5/292
378/378 [==============================] - 39s 104ms/step - loss: 4.0146 - accuracy: 0.2281
Epoch 6/292
378/378 [==============================] - 38s 101ms/step - loss: 3.9257 - accuracy: 0.2343
Epoch 7/292
378/378 [==============================] - 39s 103ms/step - loss: 3.8351 - accuracy: 0.2405
Epoch 8/292
378/378 [==============================] - 40s 105ms/step - loss: 3.7446 - accuracy: 0.2492
Epoch 9/292
378/378 [==============================] - 38s 102ms/step - loss: 3.6594 - accuracy: 0.2584
Epoch 10/292
378/378 [==============================] - 41s 109ms

378/378 [==============================] - 41s 108ms/step - loss: 1.3325 - accuracy: 0.6657
Epoch 80/292
378/378 [==============================] - 41s 107ms/step - loss: 1.3159 - accuracy: 0.6673
Epoch 81/292
378/378 [==============================] - 40s 106ms/step - loss: 1.2964 - accuracy: 0.6738
Epoch 82/292
378/378 [==============================] - 43s 115ms/step - loss: 1.2807 - accuracy: 0.6781
Epoch 83/292
378/378 [==============================] - 41s 108ms/step - loss: 1.2588 - accuracy: 0.6828
Epoch 84/292
378/378 [==============================] - 40s 107ms/step - loss: 1.2504 - accuracy: 0.6834
Epoch 85/292
378/378 [==============================] - 41s 107ms/step - loss: 1.2352 - accuracy: 0.6864
Epoch 86/292
378/378 [==============================] - 40s 106ms/step - loss: 1.2232 - accuracy: 0.6899
Epoch 87/292
378/378 [==============================] - 42s 110ms/step - loss: 1.2089 - accuracy: 0.6932
Epoch 88/292
378/378 [==============================] - 41s 109ms/st

378/378 [==============================] - 40s 105ms/step - loss: 0.4293 - accuracy: 0.8865
Epoch 157/292
378/378 [==============================] - 39s 103ms/step - loss: 0.4543 - accuracy: 0.8785
Epoch 158/292
378/378 [==============================] - 39s 103ms/step - loss: 0.4203 - accuracy: 0.8900
Epoch 159/292
378/378 [==============================] - 39s 104ms/step - loss: 0.3705 - accuracy: 0.9070
Epoch 160/292
378/378 [==============================] - 39s 103ms/step - loss: 0.3596 - accuracy: 0.9107
Epoch 161/292
378/378 [==============================] - 39s 103ms/step - loss: 0.3475 - accuracy: 0.9144
Epoch 162/292
378/378 [==============================] - 39s 104ms/step - loss: 0.3786 - accuracy: 0.9025
Epoch 163/292
378/378 [==============================] - 39s 103ms/step - loss: 0.3756 - accuracy: 0.9022
Epoch 164/292
378/378 [==============================] - 39s 104ms/step - loss: 0.3653 - accuracy: 0.9054
Epoch 165/292
378/378 [==============================] - 39s

378/378 [==============================] - 47s 124ms/step - loss: 0.0453 - accuracy: 0.9951
Epoch 234/292
378/378 [==============================] - 48s 127ms/step - loss: 0.0339 - accuracy: 0.9973
Epoch 235/292
378/378 [==============================] - 48s 128ms/step - loss: 0.0298 - accuracy: 0.9980
Epoch 236/292
378/378 [==============================] - 50s 131ms/step - loss: 0.0285 - accuracy: 0.9982
Epoch 237/292
378/378 [==============================] - 51s 135ms/step - loss: 0.0259 - accuracy: 0.9983
Epoch 238/292
378/378 [==============================] - 49s 129ms/step - loss: 0.1907 - accuracy: 0.9501
Epoch 239/292
378/378 [==============================] - 47s 124ms/step - loss: 0.6436 - accuracy: 0.8273
Epoch 240/292
378/378 [==============================] - 49s 129ms/step - loss: 0.2311 - accuracy: 0.9334
Epoch 241/292
378/378 [==============================] - 48s 126ms/step - loss: 0.1085 - accuracy: 0.9745
Epoch 242/292
378/378 [==============================] - 47s

In [135]:
# Saving the model and tokenizer
model.save('epochBG3.h5')
dump(tokenizer, open('epochBG3', 'wb'))

# Generating new tweets

In [146]:
def gen_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text
    
    for i in range(num_gen_words):
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        output_text.append(pred_word)
        
    return ' '.join(output_text)

In [153]:
model = load_model('epochBG2.h5')
tokenizer = load(open('epochBG2','rb'))

In [156]:
random.seed(os.urandom(1337))
random_pick = random.randint(0,len(text_sequences))

random_seed_text = text_sequences[random_pick]
seed_text = ' '.join(random_seed_text)
print('Seed Text: ',seed_text)

Seed Text:  pelo movimento acorda brasil ouvindo as demandas de diversos movimentos ' ' deputados aprovam acordos de cooperação tributária e defesa entre brasil e eua  '


In [157]:
gen_txt = gen_text(model, tokenizer, train_len, seed_text=seed_text, num_gen_words=31)

print(seed_text,'\n', gen_txt)

pelo movimento acorda brasil ouvindo as demandas de diversos movimentos ' ' deputados aprovam acordos de cooperação tributária e defesa entre brasil e eua  ' 
 ' e não deixar bem claro sem registrar a relação ao que ter isso pelo anuncio tomada de alguns sem resposta contra e não trabalha ' ' coincidencias ambos nada e
